In [1]:
import json
import os
import cv2
from collections import defaultdict
import re

def natural_sort_key(s):
    return [int(c) if c.isdigit() else c.lower() for c in re.split(r'(\d+)', s)]

def load_json_files(base_directory):
    merged_data = {
        'images': [],
        'annotations': [],
        'categories': []
    }
    image_id_mapping = {}
    annotation_id_counter = 0
    global_image_id_counter = 0

    # Sort folders to ensure consistent processing
    folders = sorted([f for f in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, f))],
                     key=natural_sort_key)

    for folder_name in folders:
        folder_path = os.path.join(base_directory, folder_name)

        # Find JSON file in the folder
        json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
        if not json_files:
            print(f"No JSON file found in folder: {folder_name}")
            continue

        # Use the first JSON file found
        json_file = json_files[0]
        json_path = os.path.join(folder_path, json_file)

        try:
            with open(json_path, 'r') as f:
                data = json.load(f)

            # Update image IDs and add to merged data
            for image in data['images']:
                old_id = image['id']
                new_id = global_image_id_counter
                image_id_mapping[old_id] = new_id
                image['id'] = new_id
                image['folder'] = folder_name  # Add folder information
                merged_data['images'].append(image)
                global_image_id_counter += 1

            # Update annotation IDs and image IDs, then add to merged data
            for annotation in data['annotations']:
                annotation['id'] = annotation_id_counter
                annotation_id_counter += 1
                annotation['image_id'] = image_id_mapping[annotation['image_id']]
                merged_data['annotations'].append(annotation)

            # Add categories if not already present
            for category in data['categories']:
                if category not in merged_data['categories']:
                    merged_data['categories'].append(category)

        except Exception as e:
            print(f"Error processing JSON in folder {folder_name}: {e}")

    return merged_data

def associate_frames_with_videos(json_data, base_directory):
    video_frames = {}
    total_frames = 0

    # Sort folders to ensure consistent processing
    folders = sorted([f for f in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, f))],
                     key=natural_sort_key)

    for folder_name in folders:
        folder_path = os.path.join(base_directory, folder_name)

        # Find video files in the folder
        video_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.mp4', '.avi', '.mov'))]
        video_files.sort(key=natural_sort_key)  # Sort video files using natural sort

        for video_file in video_files:
            video_path = os.path.join(folder_path, video_file)
            try:
                cap = cv2.VideoCapture(video_path)
                frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                cap.release()

                video_frames[video_path] = frame_count
                total_frames += frame_count
            except Exception as e:
                print(f"Error processing video {video_path}: {e}")

    print(f"Total frames across all videos: {total_frames}")
    print(f"Number of images in JSON: {len(json_data['images'])}")

    image_to_video = {}
    video_frame_to_image = defaultdict(dict)
    frame_index = 0

    for video_path, frame_count in video_frames.items():
        print(f"Processing {video_path} with {frame_count} frames")
        for i in range(frame_count):
            if frame_index < len(json_data['images']):
                image_id = json_data['images'][frame_index]['id']
                image_to_video[image_id] = video_path
                video_frame_to_image[video_path][i] = image_id
                frame_index += 1
            else:
                print(f"Warning: Ran out of images in JSON data while processing video {video_path}")
                break

    return image_to_video, video_frame_to_image

def process_annotations(json_data, image_to_video):
    video_annotations = defaultdict(list)
    for annotation in json_data['annotations']:
        image_id = annotation['image_id']
        video_path = image_to_video.get(image_id)
        if video_path:
            video_annotations[video_path].append(annotation)
        else:
            print(f"Warning: No video found for image_id {image_id}")

    for video_path, annotations in video_annotations.items():
        print(f"Video: {video_path}, Annotations: {len(annotations)}")

    return video_annotations

def draw_bounding_boxes(frame, annotations, categories):
    for annotation in annotations:
        bbox = annotation.get('bbox')
        category_id = annotation.get('category_id')
        category = next((cat['name'] for cat in categories if cat['id'] == category_id), 'Unknown')

        if bbox:
            x, y, w, h = [int(coord) for coord in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, category, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

def visualize_annotations(video_annotations, json_data, video_frame_to_image):
    video_paths = sorted(list(video_annotations.keys()), key=lambda x: natural_sort_key(os.path.basename(x)))
    current_video_index = 0

    window_name = 'Video Annotation Viewer'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    while current_video_index < len(video_paths):
        video_path = video_paths[current_video_index]
        annotations = video_annotations[video_path]
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"Error opening video file: {video_path}")
            current_video_index += 1
            continue

        frame_index = 0
        paused = False

        while True:
            if not paused:
                ret, frame = cap.read()
                if not ret:
                    break

                # Get annotations for this frame
                image_id = video_frame_to_image[video_path].get(frame_index)
                if image_id is not None:
                    frame_annotations = [ann for ann in annotations if ann['image_id'] == image_id]
                else:
                    frame_annotations = []

                # Draw bounding boxes
                draw_bounding_boxes(frame, frame_annotations, json_data['categories'])

            # Display the frame
            cv2.setWindowTitle(window_name, f'Video: {os.path.basename(video_path)} (Frame: {frame_index})')
            cv2.imshow(window_name, frame)

            # Control playback
            key = cv2.waitKey(0 if paused else 25) & 0xFF
            if key == ord('q'):
                print("Quitting the program.")
                cap.release()
                cv2.destroyAllWindows()
                return
            elif key == ord('n'):
                print("Moving to next video.")
                break
            elif key == ord('p'):
                print("Moving to previous video.")
                current_video_index = max(0, current_video_index - 2)
                break
            elif key == ord(' '):
                paused = not paused
                print("Paused" if paused else "Resumed")
            elif key == ord('f'):
                frame_index = min(frame_index + 10, int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1)
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
            elif key == ord('b'):
                frame_index = max(0, frame_index - 10)
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)

            if not paused:
                frame_index += 1

        cap.release()
        print(f"Finished processing video: {video_path}")
        current_video_index += 1

    cv2.destroyAllWindows()

def main():
    # Update these paths as needed
    video_directory = r'your/path/directory' #Change Path here

    # Load merged JSON data from all folders
    json_data = load_json_files(video_directory)

    # Associate frames with videos
    image_to_video, video_frame_to_image = associate_frames_with_videos(json_data, video_directory)

    # Process annotations
    video_annotations = process_annotations(json_data, image_to_video)

    # Visualize annotations
    visualize_annotations(video_annotations, json_data, video_frame_to_image)

if __name__ == "__main__":
    main()

Total frames across all videos: 8754
Number of images in JSON: 8754
Processing C:\Users\Renz\Desktop\try\1\Ks (1).mp4 with 1000 frames
Processing C:\Users\Renz\Desktop\try\1\Ks (2).mp4 with 205 frames
Processing C:\Users\Renz\Desktop\try\1\Ks (3).mp4 with 117 frames
Processing C:\Users\Renz\Desktop\try\1\Ks (4).mp4 with 207 frames
Processing C:\Users\Renz\Desktop\try\1\Ks (5).mp4 with 191 frames
Processing C:\Users\Renz\Desktop\try\2\Ks (6).mp4 with 217 frames
Processing C:\Users\Renz\Desktop\try\2\Ks (7).mp4 with 129 frames
Processing C:\Users\Renz\Desktop\try\2\Ks (8).mp4 with 85 frames
Processing C:\Users\Renz\Desktop\try\2\Ks (9).mp4 with 102 frames
Processing C:\Users\Renz\Desktop\try\2\Ks (10).mp4 with 43 frames
Processing C:\Users\Renz\Desktop\try\3\Ks (11).mp4 with 87 frames
Processing C:\Users\Renz\Desktop\try\3\Ks (12).mp4 with 275 frames
Processing C:\Users\Renz\Desktop\try\3\Ks (13).mp4 with 71 frames
Processing C:\Users\Renz\Desktop\try\3\Ks (14).mp4 with 155 frames
Proces

Finished processing video: C:\Users\Renz\Desktop\try\4\Ks (18).mp4
Quitting the program.
